In [1]:
import pandas as pd
import os
from datetime import date
import numpy as np

"""
El proposito de este script es poder generar los mercados a partir del archivo de cid latina

Para el armado de los mercados, hay parametros importantes, el mercado (tiene sus clases terapeuticas, marcas, presentaciones),
el mes y el año que se deben indicar en las variables que se encuentran debajo

Tambien es necesario guardar los archivos respetando el formato para que el codigo pueda funcionar
"""

año = '2022'
mes = '12'

direc_noe = 'C:/Users/rvozzi/genommalabinternacional/Noelia Soledad Riccelli - Plan BI/'
direc = direc_noe + 'archivos mes/' + año + mes 

fechas = pd.date_range(mes + '-' +'01-' +  str(int(año[2:])-2),mes + '-' + '01-' + año[2:], 
                         freq='MS').strftime("%Y-%m").tolist()
fechas

['2020-12',
 '2021-01',
 '2021-02',
 '2021-03',
 '2021-04',
 '2021-05',
 '2021-06',
 '2021-07',
 '2021-08',
 '2021-09',
 '2021-10',
 '2021-11',
 '2021-12',
 '2022-01',
 '2022-02',
 '2022-03',
 '2022-04',
 '2022-05',
 '2022-06',
 '2022-07',
 '2022-08',
 '2022-09',
 '2022-10',
 '2022-11',
 '2022-12']

# 1. Descarga de archivo de cid Latina

In [2]:
%%time

#ver si las fechas hay que invertirlas
v_pres_names = ['cpa', 'pres_cod']
v_pres_names.extend(fechas)
cols_list_u = [2,6]
cols_list_m = [2,6]
cols_list_u.extend(range(7,32,1))
cols_list_m.extend(range(32,57,1))

#descargo el archivo de Mercados de Unidades
v_pres_u = pd.read_csv(direc + '/Mercados/Venta_Pres.txt', header = None, names = v_pres_names, encoding='latin-1', 
                     sep=",",
                     skiprows = 1, 
                     usecols = cols_list_u)
v_pres_u['Tipo'] = 'Cantidad'

#descargo el archivo de Mercados de Montos
v_pres_m = pd.read_csv(direc + '/Mercados/Venta_Pres.txt', header = None, names = v_pres_names, encoding='latin-1', 
                     sep=",",
                     skiprows = 1, 
                     usecols = cols_list_m)
v_pres_m['Tipo'] = 'Monto'

#descargo mi maestro
maestro = pd.read_excel(direc_noe + 'Maestros/Tbl_mercados_submarcas vchiqui.xlsx', usecols='A:H')

C:\Users\rvozzi\anaconda3\lib\site-packages\IPython\core\magics\execution.py:1324: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


Wall time: 2min 38s


In [3]:
maestro

,marca_codigo,pres_cod,pres_desc,Mercado,Marca,Submarca,lab,Segmento
0,20116,2011601,ACEITE ESMERALDA MOONE linim.x 70 ml,BENGUE,OTROS,OTROS,OTROS,OTROS
1,20116,2011602,ACEITE ESMERALDA MOONE spray x 100 ml,BENGUE,OTROS,OTROS,OTROS,OTROS
2,20231,2023101,ACTRON RAPIDA ACCION 10 caps.blandas.x 400mg.,TAFIROL,ACTRON,ACTRON RAPIDA ACCION,Bayer Consumer,PLUS
3,20231,2023102,ACTRON RAPIDA ACCION 20 caps.blandas.x 400mg.,TAFIROL,ACTRON,ACTRON RAPIDA ACCION,Bayer Consumer,PLUS
4,20232,2023201,ACTRON 600 RAPIDA ACCION 10 caps.blandas.,TAFIROL,ACTRON,ACTRON 600 RAPIDA ACCION,Bayer Consumer,PLUS
...,...,...,...,...,...,...,...,...
2368,65766,9012558,COLAGENO SILUETT chocolate pote x 450 g,X-RAY,SILUETT,SILUETT COLAGENO,Pharmamerican S.,OTROS
2369,38460,9012509,TIO NACHO acond.cel.madre V2 x 415,PERSONAL CARE,TIO NACHO,TIO NACHO,Genomma Lab.,OTROS
2370,38460,9012510,TIO NACHO acond.cel.madre V2 x 200,PERSONAL CARE,TIO NACHO,TIO NACHO,Genomma Lab.,OTROS
2371,38460,9012511,TIO NACHO sh.cel.madre V2 x 415 ml,PERSONAL CARE,TIO NACHO,TIO NACHO,Genomma Lab.,OTROS


In [4]:
%%time

#esto lo hago por que hay marcas duplicadas en ambos archivos, pero en algunos casos no estan
#completamente duplicados como para hacer un drop_duplicates
v_pres_u = v_pres_u[v_pres_u['pres_cod'].isin(maestro[~maestro['Mercado'].isin(['PERSONAL CARE'])]['pres_cod'])]
v_pres_m = v_pres_m[v_pres_m['pres_cod'].isin(maestro[~maestro['Mercado'].isin(['PERSONAL CARE'])]['pres_cod'])]

#el melt hace un unpivot de los archivos
data = pd.melt(pd.concat([v_pres_u, v_pres_m,]), 
                  id_vars = ['cpa', 'pres_cod', 'Tipo'],
                  value_vars = fechas,
                  value_name = 'Valor', var_name = 'Año_Mes')

#saco los meses que tienen ceros ya que no aportan informacion
data = data[~data['Valor'].isin([0])]

#Desagrupo la columna valor segun si refleja Monto o Cantidad que sale de la columna Tipo
data = data.pivot(index = ['cpa', 'pres_cod', 'Año_Mes'],
                  columns = 'Tipo',
                  values = 'Valor')
data = data.reset_index()

#Hago un join de la tabla con el maestro
data = pd.merge(data,
                maestro,
                left_on = 'pres_cod',
                right_on = 'pres_cod',
                how = 'inner')

#agrupar pres_desc por otros
data.loc[data['Marca']=='OTROS', 'pres_desc'] = 'OTROS'
data = data.groupby(['Mercado','cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento','Año_Mes'],
                    as_index = False)['Cantidad', 'Monto'].sum()

#Limpio los cpas
data['cpa'] = data['cpa'].astype("string")
data['cpa'] = data['cpa'].map(lambda x: x[-4:] if x[0:4]=='0000' else x)
data['cpa'] = data['cpa'].astype("string")

#solo me quedo con los cpas alfanumericos ya que hay cpas del formato '1009' que agrupan varias farmacias pequeñas
mask = (data['cpa'].str.len() == 8)
data = data.loc[mask]

<timed exec>:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Wall time: 9min 30s


In [5]:
data

,Mercado,cpa,Marca,Submarca,pres_desc,lab,Segmento,Año_Mes,Cantidad,Monto
208381,BENGUE,A4126AHA,ATOMO DESIFLAMANTE,ATOMO DESIFLAMANTE,ATOMO DESINF clas.pote x 110 g,Imvi,ANALGESICOS TOPICOS,2022-11,3,2853
208382,BENGUE,A4126AHA,BENGUE,BENGUE,BENGUE DESINFLAMANTE POMADA X 150 Gr,Genomma Lab.,ANALGESICOS TOPICOS,2022-11,1,1034
208383,BENGUE,A4126AHA,BENGUE,BENGUE,BENGUE DESINFLAMANTE pomada x 65 g,Genomma Lab.,ANALGESICOS TOPICOS,2022-11,1,581
208384,BENGUE,A4126AHA,VOLTAREM,VOLTAREN EMULGEL,VOLTAREN EMULGEL pomo x 30 g,GlaxoSmithKline,ANALGESICOS TOPICOS,2022-11,1,781
208385,BENGUE,A4126AHE,ANAFLEX,ANAFLEX,ANAFLEX cr.x 70 g,Bago,ANALGESICOS TOPICOS,2022-03,1,597
...,...,...,...,...,...,...,...,...,...,...
32731774,X-RAY,Z9408ACO,INFILTREX,INFILTREX,INFILTREX comp.rec. x 30,Elea,OTROS,2022-03,3,3950
32731775,X-RAY,Z9408ACO,INFILTREX,INFILTREX,INFILTREX comp.rec. x 30,Elea,OTROS,2022-12,3,5779
32731776,X-RAY,Z9408ACO,MAXIOSTENIL PLUS,MAXIOSTENIL PLUS,MAXIOSTENIL PLUS jga.prell.x 2 ml,Trb-Pharma,OTROS,2022-10,1,9808
32731777,X-RAY,Z9408ACO,MAXIOSTENIL PLUS,MAXIOSTENIL PLUS,MAXIOSTENIL PLUS jga.prell.x 2 ml,Trb-Pharma,OTROS,2022-12,1,10731


In [6]:
#cargo la tabla entera en un archivo csv llamado base
data.to_csv('C:/Users/rvozzi/OneDrive - genommalabinternacional/Documentos/ECD/Seminario/TP' + '/base.csv',
                    sep = ',', index = False)

In [7]:
data.groupby(['Año_Mes'])['Cantidad'].sum()

Año_Mes
2020-12     8635078
2021-01     8929828
2021-02     8785120
2021-03    11202942
2021-04    11640956
2021-05    11548535
2021-06    11744757
2021-07    10765872
2021-08    10708761
2021-09    11132917
2021-10     9822923
2021-11    10618607
2021-12    10526858
2022-01    15460944
2022-02    10415705
2022-03    12913094
2022-04    11358766
2022-05    13312639
2022-06    12402574
2022-07    11572647
2022-08    11216713
2022-09    11449018
2022-10    10374235
2022-11    11507559
2022-12    10322425
Name: Cantidad, dtype: int64